Installing packages


In [ ]:
!pip install statsmodels
#import needed packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from google.colab import drive
#mounting my google driving to access data
drive.mount('/content/drive')

Mounted at /content/drive


Creatings Pandas data frames

In [ ]:
#converting the csv synthetic_data to a pandas data frame
df = pd.read_csv('/content/drive/MyDrive/Second Year Project/Student Data /2021-2228_anon_cleaned.csv')

#converting the xlsx data to a pandas data frame
df2 = pd.read_excel("/content/drive/MyDrive/Second Year Project/Student Data /Master LA _ Mentor Database.xlsx")
df2 = df2.rename(columns={'term': 'Term'})

<ipython-input-2-07481f5bd3e4>:2: DtypeWarning: Columns (7,8,9,10,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Second Year Project/Student Data /2021-2228_anon_cleaned.csv')


Below I am filtering the data to only look at coures starting in the Fall of 2013 and excluding Summmer terms.

In [ ]:
df = df[(df['Term'] >= 2131) & (df['Term'] != 2133)]
df = df[df['Term'] % 10 != 8]
df = df[(2131 <= df["Cohort Term"]) & (df["Cohort Term"] <= 2161)]




Below I am filtering out all classes containing a lab and all classes not taken at the UNI main campus.

In [ ]:
df["SYG"] = 0
df.loc[(df["Degree Term"] < df["Cohort Term"]+60) & ((df["Degree Term"] >= df["Cohort Term"])), "SYG"] = 1
print('Overall SYG rate:', len(df[df['SYG'] == 1]) / len(df['SYG']))



Overall SYG rate: 0.8377085850924331


I am narrowing down the data to only look at University Physics I and II which I will call UP1 and UP2 moving forward. I am also getting rid of all Labs, getting rid of classes not taken at the Rochester campus, and last I am getting rid of all classes taken since the start of the pandemic to account for the change in the grading system that took place during.

In [ ]:
df = df[(df['subject'] == 'PHYS') & ((df['catalog_nbr'] == '211') | (df['catalog_nbr'] == '212'))]
df = df[~(df['section Type'] == 'LAB')]
df = df[(df['class_section'] != '600') & (df['class_section'] != '601')]
# df = df[df['Term'] <= 2195]

Below I am calcualting the  six year grad rates for all students in UP1 and UP2

In [ ]:
print('UP1 and UP2 Overall SYG rate:',len(df[df['SYG'] == 1]) / len(df['SYG']))

UP1 and UP2 Overall SYG rate: 0.8454125677574784


Adding in Learning Assistant Column where 1 is a LA and 0 is no LA

In [ ]:
#creating a function to find all of the
def LA_Finder(subject, catalog_nbr, class_section, Term, df):
    df.loc[(df['Term'] == Term) & (df['subject'] == subject) &
           (df['class_section'] == class_section) & (df['catalog_nbr'] == catalog_nbr), 'LA'] = 1
    return df

df['LA'] = 0

# looping over alll the rows in the LA dataframe
for index, row in df2.iterrows():
    subject = row['subject']
    catalog_nbr = row['catalog_nbr']
    class_section = row['class_section']
    Term = row['Term']
    df = LA_Finder(subject, catalog_nbr, class_section, Term, df)


Below I am calculating the SYG rate for students who had an LA at some points in UP1 OR UP2.

In [ ]:
LA_df = df[df['LA'] == 1]
print('UP1 and UP2 LA Overall SYG rate with LA :',len(LA_df[LA_df['SYG'] == 1]) / len(LA_df['SYG']))

NO_LA_df = df[df['LA'] == 0]
print('UP1 and UP2 NO LA Overall SYG rate:',len(NO_LA_df[NO_LA_df['SYG'] == 1]) / len(NO_LA_df['SYG']))


UP1 and UP2 LA Overall SYG rate with LA : 0.8468354430379746
UP1 and UP2 NO LA Overall SYG rate: 0.8447515436636284


Adding in binary columns to capture demographics

In [ ]:
# Create a new column 'gender dummy' with 1 for male or 0 for female/other
df['gender dummy'] = df['gender'].apply(lambda x: 1 if x in ['F', 'U'] else 0)

# Create a new column 'First Gen dummy' with 1 for First Generation students or 0 for else
df['First Gen dummy'] = df['First Generation'].apply(lambda x: 1 if x == 'First Gen' else 0)
# Fill NaN values with 0
df['First Generation'] = df['First Generation'].fillna(0)

# Create a new column 'AALANA dummy' with 1 for AALANA or 0 for else
df['AALANA dummy'] = df['AALANA'].apply(lambda x: 1 if x == 'AALANA' else 0)

# Create a new column 'DFW dummy' with 1 for 'D' or 'F' or 'W' grade or lower or 0 for else
df['DFW dummy'] = df['Grade'].apply(lambda x: 1 if x in ['D', 'F', 'W'] else 0)


Below I am calculating the DFW rate for students who took either UP1 or UP2 with/without an LA

In [ ]:
LA_df = df[df['LA'] == 1]
print('UP1 and UP2 with LA Overall DFW rate:',len(LA_df[LA_df['DFW dummy'] == 1]) / len(LA_df['DFW dummy']))

NO_LA_df = df[df['LA'] == 0]
print('UP1 and UP2 NO LA Overall DFW rate:',len(NO_LA_df[NO_LA_df['DFW dummy'] == 1]) / len(NO_LA_df['DFW dummy']))

UP1 and UP2 with LA Overall DFW rate: 0.19556962025316454
UP1 and UP2 NO LA Overall DFW rate: 0.2502205233754778


Below I am creating 4 new columns to capture the follwing informaiton


1. UP1 LA Only: The student took UP1 with an LA but not UP2 with an LA
2. UP2 LA Only: The student took UP2 with an LA but not UP1 with an LA
3. UP1&2 No LA: The student took neither UP1 or UP2 with an LA
4. UP1&2 LA: The student took both UP1 and UP2 with an LA




In [ ]:
# Update values based on condition.
IDs_UP1_LA = df[(df['catalog_nbr'] == '211') & (df['LA'] == 1)]['ID'] # all people who had LA in UP1
IDs_UP2_LA = df[(df['catalog_nbr'] == '212') & (df['LA'] == 1)]['ID']# all people who had LA in UP2
IDs_UP1_notUP2_LA = set.difference(set(IDs_UP1_LA), set(IDs_UP2_LA))
IDs_UP2_notUP1_LA = set.difference(set(IDs_UP2_LA), set(IDs_UP1_LA))
IDs_UP1_UP2_LA = set.intersection(set(IDs_UP2_LA), set(IDs_UP1_LA))

IDs_neither = set.difference(set(df["ID"].unique()), set.union( set(IDs_UP2_LA), set(IDs_UP1_LA)))

df['UP1 LA Only'] = 0
df.loc[df['ID'].isin(IDs_UP1_notUP2_LA), 'UP1 LA Only'] = 1

df['UP2 LA Only'] = 0
df.loc[df['ID'].isin(IDs_UP2_notUP1_LA), 'UP2 LA Only'] = 1

df['UP1&2 No LA'] = 0
df.loc[df['ID'].isin(IDs_neither), 'UP1&2 No LA'] = 1
#df.loc[df['ID'] == 0, 'UP1&2 No LA'] = 1

df['UP1&2 LA'] = 0
df.loc[df['ID'].isin(IDs_UP1_UP2_LA), 'UP1&2 LA'] = 1

In [ ]:
# Print value counts in a nice way
print("UP1 LA Only:\n", df['UP1 LA Only'].value_counts())
print("\nUP2 LA Only:\n", df['UP2 LA Only'].value_counts())
print("\nUP1&2 LA:\n", df['UP1&2 LA'].value_counts())
print("\nUP1&2 No LA:\n", df['UP1&2 No LA'].value_counts())


UP1 LA Only:
 UP1 LA Only
0    3708
1    1273
Name: count, dtype: int64

UP2 LA Only:
 UP2 LA Only
0    4099
1     882
Name: count, dtype: int64

UP1&2 LA:
 UP1&2 LA
0    4611
1     370
Name: count, dtype: int64

UP1&2 No LA:
 UP1&2 No LA
0    2525
1    2456
Name: count, dtype: int64


Below I am calcualting the SYG groups based on the groups above

In [ ]:
# Set initial values to 0
df['UP1 LA Only SYG'] = 0
df['UP2 LA Only SYG'] = 0
df['UP1&2 LA SYG'] = 0
df['UP1&2 No LA SYG'] = 0

# Update values based on conditions
df.loc[(df['UP1 LA Only'] == 1) & (df['SYG'] == 1), 'UP1 LA Only SYG'] = 1
df.loc[(df['UP2 LA Only'] == 1) & (df['SYG'] == 1), 'UP2 LA Only SYG'] = 1
df.loc[(df['UP1&2 LA'] == 1) & (df['SYG'] == 1), 'UP1&2 LA SYG'] = 1
df.loc[(df['UP1&2 No LA'] == 1) & (df['SYG'] == 1), 'UP1&2 No LA SYG'] = 1


print(df['UP1 LA Only SYG'].value_counts())
print(df['UP2 LA Only SYG'].value_counts())
print(df['UP1&2 LA SYG'].value_counts())
print(df['UP1&2 No LA SYG'].value_counts())

GR_UP1_NoUP2_LA = len(df[df['UP1 LA Only SYG'] == 1]) / len(df[df['UP1 LA Only'] == 1])
GR_UP2_No_UP1_LA = len(df[df['UP2 LA Only SYG'] == 1]) / len(df[df['UP2 LA Only'] == 1])
GR_UP2_And_UP1_LA = len(df[df['UP1&2 LA SYG'] == 1]) / len(df[df['UP1&2 LA'] == 1])
GR_UP2_And_UP1_No_LA = len(df[df['UP1&2 No LA SYG'] == 1]) / len(df[df['UP1&2 No LA'] == 1])

print(GR_UP1_NoUP2_LA)
print(GR_UP2_No_UP1_LA)
print(GR_UP2_And_UP1_LA)
print(GR_UP2_And_UP1_No_LA)

UP1 LA Only SYG
0    3955
1    1026
Name: count, dtype: int64
UP2 LA Only SYG
0    4199
1     782
Name: count, dtype: int64
UP1&2 LA SYG
0    4667
1     314
Name: count, dtype: int64
UP1&2 No LA SYG
0    2892
1    2089
Name: count, dtype: int64
0.8059701492537313
0.8866213151927438
0.8486486486486486
0.8505700325732899


In [ ]:
# Set initial values to 0
df['UP1 LA Only DFW'] = 0
df['UP2 LA Only DFW'] = 0
df['UP1&2 LA DFW'] = 0
df['UP1&2 No LA DFW'] = 0

# Update values based on conditions
df.loc[(df['UP1 LA Only'] == 1) & (df['DFW dummy'] == 1), 'UP1 LA Only DFW'] = 1
df.loc[(df['UP2 LA Only'] == 1) & (df['DFW dummy'] == 1), 'UP2 LA Only DFW'] = 1
df.loc[(df['UP1&2 LA'] == 1) & (df['DFW dummy'] == 1), 'UP1&2 LA DFW'] = 1
df.loc[(df['UP1&2 No LA'] == 1) & (df['DFW dummy'] == 1), 'UP1&2 No LA DFW'] = 1


print(df['UP1 LA Only DFW'].value_counts())
print(df['UP2 LA Only DFW'].value_counts())
print(df['UP1&2 LA DFW'].value_counts())
print(df['UP1&2 No LA DFW'].value_counts())


DFWR_UP1_NoUP2_LA = len(df[df['UP1 LA Only DFW'] == 1]) / len(df[df['UP1 LA Only'] == 1])
DFWR_UP2_No_UP1_LA = len(df[df['UP2 LA Only DFW'] == 1]) / len(df[df['UP2 LA Only'] == 1])
DFWR_UP2_And_UP1_LA = len(df[df['UP1&2 LA DFW'] == 1]) / len(df[df['UP1&2 LA'] == 1])
DFWR_UP2_And_UP1_No_LA = len(df[df['UP1&2 No LA DFW'] == 1]) / len(df[df['UP1&2 No LA'] == 1])

print(DFWR_UP1_NoUP2_LA)
print(DFWR_UP2_No_UP1_LA)
print(DFWR_UP2_And_UP1_LA)
print(DFWR_UP2_And_UP1_No_LA)

UP1 LA Only DFW
0    4712
1     269
Name: count, dtype: int64
UP2 LA Only DFW
0    4755
1     226
Name: count, dtype: int64
UP1&2 LA DFW
0    4874
1     107
Name: count, dtype: int64
UP1&2 No LA DFW
0    4423
1     558
Name: count, dtype: int64
0.211311861743912
0.2562358276643991
0.2891891891891892
0.2271986970684039


In [ ]:
# Assuming df is your DataFrame
min_term = df['Term'].min()
max_term = df['Term'].max()

print(f'The range of values in the Term column is [{min_term}, {max_term}]')
# Fall Semester 2013 - to Summer Semester 2023

The range of values in the Term column is [2131.0, 2215.0]


In [ ]:

F14_Cohort = set(df[df["Cohort Term"] == 2141]["ID"])
print(f'There are {len(F14_Cohort)} students in the Fall 2014 Cohort.' )
#calculating students from the years up to 2020.
Grad_Range = set(df[df['Degree Term'] <= 2205]["ID"])

#Now
# Tot_Grad14 = F14_Cohort.intersection(Grad_Range)
# print(f' The total number of students earning a degree within six years of 2014 was {len(Tot_Grad14)}. ')
# #df[df['Degree Term'] <= 2202].value_counts()
# print(f' The 6-year graduation rate estimation for the 2014 cohort is {len(Tot_Grad14)/len(F14_Cohort)}.')

There are 767 students in the Fall 2014 Cohort.


In [ ]:
# # Rename the 'gender' column to 'Gender'
# df = df.rename(columns={'gender': 'Gender'})
# # Map values in the 'Gender' column
# gender_mapping = {0: 'Male', 1: 'Non Male'}
# df['gender dummy'] = df['gender dummy'].map(gender_mapping)

# df = df.rename(columns={'AALANA': 'Race'})
# # Map values in the 'Race' column
# df['Race'] = df['Race'].apply(lambda x: 'AALANA' if x == 'AALANA' else 'Non AALANA')

# # Rename the 'First Generation' column to 'First Gen status'
# df = df.rename(columns={'First Generation': 'First Gen status'})
# # Map values in the 'First Gen status' column
# df['First Gen status'] = df['First Gen status'].apply(lambda x: 'FG' if x == 'First Gen' else 'CG')


# # Rename the 'LA' column to 'Instructional type'
# df = df.rename(columns={'LA': 'Instruction type'})
# # Map values in the 'Instructional type' column
# df['Instruction type'] = df['Instruction type'].apply(lambda x: 'LA' if x == 1 else 'Traditional')

Below I am calculating the six-year graduation as a new a new column called SYG with a 1 if the student graduated with in six years and 0 if they did not.

In [ ]:
# # Create a cross-tabulation table
# table = pd.crosstab(index=[df['gender dummy'], df['Race'], df['First Gen status'], df['Instruction type']],
#                     columns='N',
#                     margins=True, margins_name='Total', rownames=['gender dummy', 'Race', 'First Gen status', 'Instruction type'],
#                     colnames=['N'])

# # Convert the table to LaTeX format
# latex_table = table.to_latex()

# # Print the LaTeX table
# print(latex_table)

# #print(table)


In [ ]:
#  # Set the file name
# file_name = "Logistic_Data_UP_SYG.csv"

# # Mount Google Drive
# drive.mount('/content/drive')

# # Set the path to save the CSV file in your Google Drive
# file_path = f'/content/drive/MyDrive/{file_name}'

# # Save DataFrame to CSV
# df.to_csv(file_path, index=False)

# # Display the path where the file is saved

In [ ]:
len(df)

4981